In [1]:
from transformer import Transformer

model = Transformer(10000,128,28,4,512,100,10000)
model.parameters

<bound method Module.parameters of Transformer(
  (embedding): Embedding()
  (atten_layers): ModuleList(
    (0-27): 28 x TransformerBlock(
      (mha_layer): MHALayer(
        (rms): RMSNorm()
        (mhsa): MultiHeadSelfAttention(
          (linear_q): Linear()
          (linear_k): Linear()
          (linear_v): Linear()
          (linear_o): Linear()
          (ScaledDotProductAttention): ScaledDotProductAttention(
            (sm): SoftMax()
          )
          (rope): RotaryPositionalEmbedding()
        )
      )
      (ffn_layer): FFNLayer(
        (swiglu): SwiGLU(
          (linear1): Linear()
          (linear2): Linear()
          (linear3): Linear()
          (silu): SiLU()
        )
        (rms): RMSNorm()
      )
    )
  )
  (rms): RMSNorm()
  (output_linear): Linear()
)>

In [4]:
import torch
a = torch.tensor([0,4,4])

In [5]:
a.sqrt()

tensor([0., 2., 2.])

In [6]:
a.data_ptr == a.sqrt().data_ptr

False

In [11]:
b = torch.tensor([0,4,4],dtype=torch.float32)
a = torch.nn.Parameter(data=b, requires_grad=True)

In [ ]:
a.data